In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import random as rand
import seaborn as sns
import pandas as pd
import math
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
def convertirACadena(num):
    return '{:032b}'.format(int(num))

def convEnt(num):
    return '{:05b}'.format(int(num))

def generaComplementarias(numero):
    generados = 0
    funcComp = []
    while generados < numero:
        funcRand = rand.randint(0, 2**32-1)
        if funcRand not in funciones:
            funcComp.append(funcRand)
            generados += 1
    return funcComp

def leeFunciones(nombreFichero):
    funciones = np.genfromtxt(nombreFichero, dtype = float, usecols=0, unpack=True)
    funcionesStr = np.genfromtxt(nombreFichero, dtype = str, usecols=1, unpack=True)
    return funciones, funcionesStr

def normaliza(valores, minimo, maximo):
    return (valores - minimo) /(maximo - minimo)

def puntuacionRepeticiones(segmentos):
    puntuacion = 0
    for elem in set(segmentos):
        apariciones = segmentos.count(elem)
        if apariciones > 1:
            puntuacion += apariciones*len(elem)
    return puntuacion

def repetitividad(s):
    puntRepGlobal = 0
    posActual = 0
    segmentos = []
    for num in [4,4,4,4,4,4,4,4]:
        segmentos.append(s[posActual:posActual+num])
        posActual += num
    return puntuacionRepeticiones(segmentos)

def calculaRepetitividades(funciones):
    return np.array([repetitividad(convertirACadena(s)) for s in funciones])

def cerosFuncion(funcion):
    return [i for i in range(32) if funcion[i] == '0']

def unosFuncion(funcion):
    return [i for i in range(32) if funcion[i] == '1']

def diferenciaPar(par1,par2):
    diferencias1 = ""
    diferencias2 = ""
    contexto = ""
    posiciones = []
    for i in range(len(par1)):
        if par1[i] != par2[i]:
            diferencias1 += par1[i]
            diferencias2 += par2[i]
            posiciones.append(i)
        else:
            contexto += par1[i]
    return diferencias1, diferencias2, posiciones, par1, par2, contexto

def coincidenPar(cad1,cad2):
    return sum(map(lambda x,y: x==y, cad1, cad2))

def puntuacionDistParesFuncion(funcion):
    saca0 = cerosFuncion(funcion)
    saca1 = unosFuncion(funcion)
    punt = 0
    distintosAlMenos2 = []
    for fun0 in saca0:
        for fun1 in saca1:
            dif1, dif2, pos, par1, par2, contexto = diferenciaPar(convEnt(fun0),convEnt(fun1))
            distintosAlMenos2.append((dif1,dif2,pos,par1,par2,contexto))
    cruces = 0
    for i in range(len(distintosAlMenos2)):
        dif1, dif2, pos, par1, par2, contexto = distintosAlMenos2[i]
        for j in range(i, len(distintosAlMenos2)):
            dif1bis, dif2bis, posbis, par1bis, par2bis, contextobis = distintosAlMenos2[j]
            if dif1 == dif2bis and dif2 == dif1bis and pos == posbis:
                    cruces += 5-len(dif1)
                    cruces += coincidenPar(contexto,contextobis)/len(contexto)
    return cruces

def puntuacionDistPares(funciones):
    return np.array([puntuacionDistParesFuncion(convertirACadena(funcion)) for funcion in funciones])


Repetitividad Cliqué

In [ ]:
pdClique = pd.read_csv("/content/drive/MyDrive/TFG/salidaMuestraDificilK4.csv")

In [ ]:
funcionesClique = np.array(list(map(lambda lista: ''.join(map(lambda elem: str(elem), lista)), pdClique.values)))

In [ ]:
repetitividadClique = calculaRepetitividades(funcionesClique)

In [ ]:
print('Media de repetitividad de Cliqué:', np.mean(repetitividadClique))

Media de repetitividad de Cliqué: 12.227972152010997


Repetitividad paridad

In [ ]:
pdParidad = pd.read_csv("/content/drive/MyDrive/TFG/muestrasDificilsalidaParidad.csv")

In [ ]:
funcionesParidad = np.array(list(map(lambda lista: ''.join(map(lambda elem: str(elem), lista)), pdParidad.values)))

In [ ]:
repetitividadParidad = calculaRepetitividades(funcionesParidad)

In [ ]:
print('Media de repetividad de Paridad:', np.mean(repetitividadParidad))

Media de repetividad de Paridad: 10.4638


Distinción pares Clique

In [ ]:
crucesClique = puntuacionDistPares(funcionesClique)

In [ ]:
print('Media distincion pares Clique:', np.mean(crucesClique))

Media distincion pares Clique: 629.2620652447047


Distinción pares Paridad

In [ ]:
crucesParidad = puntuacionDistPares(funcionesParidad)

In [ ]:
print('Media distincion pares Paridad:', np.mean(crucesParidad))

Media distincion pares Paridad: 600.6745250000001


Regresión logística Clique y Paridad

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/TFG/metricas")
X = df.loc[:,['Repetitividad', 'Disticion de Pares']]
Y = df.loc[:,['Clase']]
clf = LogisticRegression(random_state=333).fit(X, Y.values.ravel())

In [ ]:
y_test_predClique = clf.predict(pd.DataFrame({'Repetitividad':repetitividadClique, 'Disticion de Pares':crucesClique}))

In [ ]:
print('Respuesta "Complementario" con métricas en Cliqué:', np.sum(y_test_predClique == 'Complementario')/len(y_test_predClique)*100, '%')

Respuesta "Complementario" con métricas en Cliqué: 52.42190013746619 %


In [ ]:
y_test_predParidad = clf.predict(pd.DataFrame({'Repetitividad':repetitividadParidad, 'Disticion de Pares':crucesParidad}))

In [ ]:
print('Respuesta "Complementario" con métricas en Paridad:', np.sum(y_test_predParidad == 'Complementario')/len(y_test_predParidad)*100, '%')

Respuesta "Complementario" con métricas en Paridad: 46.42000000000001 %


Repetitividad Set Cover

In [ ]:
pdSetCover = pd.read_csv("/content/drive/MyDrive/TFG/muestrasDificilsalidaSetCoverMin1.csv")

In [ ]:
funcionesSetCover = np.array(list(map(lambda lista: ''.join(map(lambda elem: str(elem), lista)), pdSetCover.values)))

array(['00100000000010101011100110101101',
       '10101001001101000110100010110000',
       '11010111101100001010101000010111', ...,
       '00001000001101001111000100111011',
       '11110100010011100011001110100001',
       '11011001010001110110100011100000'], dtype='<U32')

In [ ]:
periodicidadesSetCover = calculaPeriodicidadesSumaMejores(funcionesSetCover, combinaciones)

In [ ]:
print('Media periodicidad y repetición de SetCover:', np.mean(periodicidadesSetCover))

Media periodicidad y repetición de SetCover: 22.445084327437268


Repetitividad Mayoría

In [ ]:
pdMayoria = pd.read_csv("/content/drive/MyDrive/TFG/muestrasDificilsalidaMayoria.csv")

In [ ]:
funcionesMayoria = np.array(list(map(lambda lista: ''.join(map(lambda elem: str(elem), lista)), pdMayoria.values)))

In [ ]:
periodicidadesMayoria = calculaPeriodicidadesSumaMejores(funcionesMayoria, combinaciones)

In [ ]:
print('Media periodicidad y repetición de mayoría:', np.mean(periodicidadesMayoria))

Media periodicidad y repetición de mayoría: 19.113752830716777


Distincion pares Set Cover

In [ ]:
crucesSetCover = dificultadCruces(funcionesSetCover, cuentaLong = True, muestra = False, usaContexto = True)
print('Media distincion pares Set Cover:', np.mean(crucesSetCover))

Media distincion pares Set Cover: 625.4670586344012


Distincion pares Mayoría

In [ ]:
crucesMayoria = dificultadCruces(funcionesMayoria, cuentaLong = True, muestra = False, usaContexto = True)
print('Media distincion pares Mayoria:', np.mean(crucesMayoria))

Media distincion pares Mayoria: 629.4473639347599
